**Sequence Classification**

In [2]:
!pip install transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 100.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.4 MB/s eta 0:00:00


In [3]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"

In [4]:
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=checkpoint)

In [5]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

In [7]:
raw_inputs = [
    "I get irritated during the winter.",
    "Ravi received the director's gold medal for being the topper.",
    "As expected, Sumana received her promotion letter today."
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)

{'input_ids': tensor([[  101,  1045,  2131, 15560,  2076,  1996,  3467,  1012,   102,     0,
             0,     0,     0,     0,     0,     0],
        [  101, 16806,  2363,  1996,  2472,  1005,  1055,  2751,  3101,  2005,
          2108,  1996,  2327,  4842,  1012,   102],
        [  101,  2004,  3517,  1010,  7680,  5162,  2363,  2014,  4712,  3661,
          2651,  1012,   102,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]])}


In [8]:
outputs = model(**inputs)
print(outputs)

SequenceClassifierOutput(loss=None, logits=tensor([[ 4.0595, -3.3044],
        [-3.9833,  4.3278],
        [-2.4997,  2.6847]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


In [9]:
print(outputs.logits.shape)

torch.Size([3, 2])


In [10]:
#Find the label / class probabilities
import torch
outputs = torch.nn.functional.softmax(outputs.logits, dim = -1)
print(outputs)

tensor([[9.9937e-01, 6.3328e-04],
        [2.4571e-04, 9.9975e-01],
        [5.5720e-03, 9.9443e-01]], grad_fn=<SoftmaxBackward0>)


We get [0.99, 0.0006] as the output for the first input, [0.0002, 0.99] as the output for the second input, and finally [0.005, 0.99] as the output for the third input sample.

Here we observe that our model is 99% confident that the first input sample belongs to the NEGATIVE class, 99% confident that the second input sample belongs to the POSITIVE class, and 99% confident that the third input sample belongs to the POSITIVE class. We observe that the model’s output is quite accurate.

We can check the labels of the model in the following way:

In [11]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}